# 뉴스 요약하기

1. Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.	분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.
2. 텍스트 요약모델이 성공적으로 학습되었음을 확인하였다.	모델 학습이 진행되면서 train loss와 validation loss가 감소하는 경향을 그래프를 통해 확인했으며, 실제 요약문에 있는 핵심 단어들이 요약 문장 안에 포함되었다.
3. Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.	두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교하고 분석 결과를 표로 정리하여 제시하였다.


In [16]:
# 라이브러리 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import os
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# 데이터 불러오기
raw_data = pd.read_csv('data/news_summary_more.csv', encoding='iso-8859-1')
print(raw_data.head())

data = raw_data.copy()

                                           headlines  \
0  upGrad learner switches to career in ML & Al w...   
1  Delhi techie wins free food from Swiggy for on...   
2  New Zealand end Rohit Sharma-led India's 12-ma...   
3  Aegon life iTerm insurance plan helps customer...   
4  Have known Hirani for yrs, what if MeToo claim...   

                                                text  
0  Saurav Kant, an alumnus of upGrad and IIIT-B's...  
1  Kunal Shah's credit card bill payment platform...  
2  New Zealand defeated India by 8 wickets in the...  
3  With Aegon Life iTerm Insurance plan, customer...  
4  Speaking about the sexual harassment allegatio...  


# 데이터 전처리

텍스트 정제 (Cleaning)
- HTML 태그, URL, 특수문자 제거
- 불필요한 공백 제거
- 이모지 제거

토큰화 (Tokenization) : 텍스트를 단어 또는 문장 단위로 분리

불용어 제거 (Stopwords Removal) : 의미가 적은 단어 제거 (예: a, an, the 등)

소문자 변환 (Lowercasing) : 모든 단어를 소문자로 변환하여 일관성 유지

어간 추출 및 표제어 추출 (Stemming and Lemmatization) : 단어의 기본 형태로 변환하여 단어 수 축소

문장 길이 조정 (Sentence Length Adjustment) : 모델에 입력할 수 있는 텍스트 길이를 조정

구두점 제거 (Punctuation Removal):구두점을 제거하여 텍스트 단순화

특수문자 제거 (Special Characters Removal):텍스트에 포함된 특수문자 제거

정규 표현식 (Regular Expressions): 특정 패턴을 찾아 변환 또는 제거

단어 임베딩 (Word Embeddings):단어를 임베딩하여 벡터화 (예: Word2Vec, GloVe 등)

패딩 (Padding):문장의 길이를 동일하게 맞추기 위해 패딩을 추가 (주로 최대 문장 길이에 맞춰 패딩 추가)

어텐션 마스크 (Attention Mask):패딩된 부분을 어텐션 메커니즘에서 무시하기 위해 마스크를 추가

In [15]:
# 중복, null값 확인 및 제거 
def find_duplicated_null(data):
    print('전체 샘플수 :', (len(data)))
    print('Text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())
    print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())
    print(data.isnull().sum())
    
    data.drop_duplicates(subset = ['text'], inplace=True) # text 열 기준 중복 제거
    data.dropna(axis=0, inplace=True)
    
    return data

data = find_duplicated_null(data)
data = find_duplicated_null(data)

전체 샘플수 : 98401
Text 열에서 중복을 배제한 유일한 샘플의 수 : 98360
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 98280
headlines    0
text         0
dtype: int64
전체 샘플수 : 98360
Text 열에서 중복을 배제한 유일한 샘플의 수 : 98360
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 98262
headlines    0
text         0
dtype: int64


In [17]:
# 텍스트 정규화(text normalization)와 불용어 제거
# 같은 의미인데도 다른 표현으로 쓰여 마치 다른 단어들처럼 간주되는 경우 (예를 들면 it'll은 it will과 같고, mustn't은 must not과 같음)

# 정규화 사전 생성
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

# 불용어 :  텍스트에는 자주 등장하지만 자연어 처리를 할 때 실질적으로 별 도움이 되지 않는 단어들 (예를 들면, I, my, me, over, 조사, 접미사 등, 의미를 분석해도 도움이 되지 않는 단어들)
# 학습 데이터에서 빈도수가 낮은 단어나 길이가 짧은 단어들을 제거하는 것이 효과적일 수 있음
# 타겟 데이터에서는 지우지 않는게 자연스러운 문장이 됨 
print('불용어 개수 :', len(stopwords.words('english') ))
print(stopwords.words('english'))

불용어 개수 : 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 's

In [18]:
# 데이터 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\<.*?\>', '', sentence) # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)  # URL 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","", sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # 불필요한 공백 제거
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # 구두점 제거
    
    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

# 확인
temp_text = 'Everything I bought was great, infact I ordered twice and the third ordered was<br />for my mother and father.'
temp_summary = 'Great way to start (or finish) the day!!!'

print("text: ", preprocess_sentence(temp_text))
print("summary:", preprocess_sentence(temp_summary, False))  # 불용어를 제거하지 않습니다.

text:  everything bought great infact ordered twice third ordered wasfor mother father
summary: great way to start the day


In [7]:
# 텍스트 정제 함수
def clean_text(text):
    text = text.lower()  # 소문자 변환
    text = re.sub(r'\<.*?\>', '', text)  # HTML 태그 제거
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # URL 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
    text = re.sub(r'\[.*?\]', '', text)  # 괄호로 묶인 내용 제거
    text = text.translate(str.maketrans('', '', string.punctuation))  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    return text

# 텍스트 정제 적용
data['cleaned_text'] = data['text'].apply(clean_text)
data['cleaned_headlines'] = data['headlines'].apply(clean_text)

# 결과 확인
print(data[['text', 'cleaned_text']].head())
print(data[['headlines', 'cleaned_headlines']].head())

                                                text  \
0  Saurav Kant, an alumnus of upGrad and IIIT-B's...   
1  Kunal Shah's credit card bill payment platform...   
2  New Zealand defeated India by 8 wickets in the...   
3  With Aegon Life iTerm Insurance plan, customer...   
4  Speaking about the sexual harassment allegatio...   

                                        cleaned_text  
0  saurav kant an alumnus of upgrad and iiitbs pg...  
1  kunal shahs credit card bill payment platform ...  
2  new zealand defeated india by  wickets in the ...  
3  with aegon life iterm insurance plan customers...  
4  speaking about the sexual harassment allegatio...  
                                           headlines  \
0  upGrad learner switches to career in ML & Al w...   
1  Delhi techie wins free food from Swiggy for on...   
2  New Zealand end Rohit Sharma-led India's 12-ma...   
3  Aegon life iTerm insurance plan helps customer...   
4  Have known Hirani for yrs, what if MeToo claim... 